In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
data_path = Path("planet", "planet")
output_path = Path("artifacts")
output_path.mkdir(exist_ok=True)

image_path = data_path / "train-jpg"
    

train_classes = pd.read_csv(data_path / "train_classes.csv")

train_classes["tag"] = (
    train_classes["tags"]
    .apply(
        lambda x: [tag for tag in x.split(" ") if tag != "selective_logging"]
    )
    .apply(
        lambda x: sorted(x)[-1]
    )
)

train_labels = {
    row["image_name"]: row["tag"] for i, row in train_classes.iterrows()
}

In [15]:
sorted(train_classes["tag"].unique())

['clear',
 'cloudy',
 'cultivation',
 'habitation',
 'haze',
 'partly_cloudy',
 'primary',
 'road',
 'slash_burn',
 'water']

In [3]:
from fastai.vision.all import *

In [4]:
def label(file_name):
    return train_labels[file_name.replace(".jpg", "")]

In [5]:
# Load exported model
learn = load_learner(output_path / "model.pkl")

In [6]:
image_name = "train_0"
img = plt.imread(data_path / "train-jpg" / f"{image_name}.jpg")[:,:,:-1]
# img = PILImage.create(img)
learn.predict(img)


('primary',
 tensor(6),
 tensor([3.4738e-04, 1.0227e-03, 6.4591e-05, 1.1950e-04, 1.0738e-04, 2.1656e-04,
         9.9546e-01, 1.2084e-03, 4.9242e-04, 9.6052e-04]))

In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
from huggingface_hub import push_to_hub_fastai

repo_id = "davidmasip/deforestation_predatathon"

push_to_hub_fastai(learner=learn, repo_id=repo_id)

'https://huggingface.co/davidmasip/deforestation_predatathon/tree/main/'

In [9]:
from huggingface_hub import from_pretrained_fastai


learner_hf = from_pretrained_fastai(repo_id)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/23.1M [00:00<?, ?B/s]

In [10]:
_, _, probs = learn.predict(img)
print(probs)


tensor([3.4738e-04, 1.0227e-03, 6.4591e-05, 1.1950e-04, 1.0738e-04, 2.1656e-04,
        9.9546e-01, 1.2084e-03, 4.9242e-04, 9.6052e-04])


In [11]:
_, _, probs = learner_hf.predict(img)
print(probs)


tensor([3.4738e-04, 1.0227e-03, 6.4591e-05, 1.1950e-04, 1.0738e-04, 2.1656e-04,
        9.9546e-01, 1.2084e-03, 4.9242e-04, 9.6052e-04])
